출처:https://docs.qiime2.org/2019.7/tutorials/moving-pictures/

# Moving Pictures tutorial

# Sample metadata

In [1]:
!wget \
  -O "sample-metadata.tsv" \
  "https://data.qiime2.org/2019.7/tutorials/moving-pictures/sample_metadata.tsv"

--2019-09-05 09:34:17--  https://data.qiime2.org/2019.7/tutorials/moving-pictures/sample_metadata.tsv
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/15dH-TIA0td8qcC-ali0g2snXG8Dl26wGMKz1Zcji6kA/export?gid=0&format=tsv [following]
--2019-09-05 09:34:17--  https://docs.google.com/spreadsheets/d/15dH-TIA0td8qcC-ali0g2snXG8Dl26wGMKz1Zcji6kA/export?gid=0&format=tsv
Resolving docs.google.com (docs.google.com)... 172.217.163.238, 2404:6800:4004:80d::200e
Connecting to docs.google.com (docs.google.com)|172.217.163.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: `sample-metadata.tsv'

    [ <=>                                   ] 2,094       --.-K/s   in 0s      

2019-09-05 09:34:18 (20.5 MB/s) - `sample-metadata.tsv' saved [

# Obtaining and importing data

Download the sequence reads that we’ll use in this analysis. In this tutorial we’ll work with a small subset of the complete sequence data so that the commands will run quickly.

In [2]:
!mkdir emp-single-end-sequences

In [3]:
!ls

emp-single-end-sequences  Moving_pictures.ipynb  sample-metadata.tsv


In [4]:
!wget \
  -O "emp-single-end-sequences/barcodes.fastq.gz" \
  "https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"

--2019-09-05 09:38:07--  https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz [following]
--2019-09-05 09:38:08--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.236.0
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.236.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3783785 (3.6M) [application/x-gzip]
Saving to: `emp-single-end-sequences/barcodes.fastq.gz'

100%[======================================>] 3,783,785   2.

In [5]:
!wget \
  -O "emp-single-end-sequences/sequences.fastq.gz" \
  "https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"

--2019-09-05 09:38:16--  https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz [following]
--2019-09-05 09:38:17--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.221.88
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.221.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25303756 (24M) [binary/octet-stream]
Saving to: `emp-single-end-sequences/sequences.fastq.gz'

100%[======================================>] 25,303,

In [6]:
!qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza

Imported emp-single-end-sequences as EMPSingleEndDirFmt to emp-single-end-sequences.qza


# Demultiplexing sequences

To demultiplex sequences we need to know which barcode sequence is associated with each sample.
  

In [7]:
!qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences demux.qza \
  --o-error-correction-details demux-details.qza

Saved SampleData[SequencesWithQuality] to: demux.qza
Saved ErrorCorrectionDetails to: demux-details.qza


After demultiplexing, it’s useful to generate a summary of the demultiplexing results.

In [8]:
!qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

Saved Visualization to: demux.qzv


In [9]:
from qiime2 import Visualization
Visualization.load('demux.qzv')

<visualization: Visualization uuid: bbd61890-d5a5-4f87-9410-37b5ba3a64b6>

# Sequence quality control and feature table construction

QIIME 2 plugins are available for several quality control methods, including DADA2, Deblur, and basic quality-score-based filtering.

## Option 1: DADA2

DADA2 is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. 

In [10]:
!qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 120 \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza

Saved FeatureTable[Frequency] to: table-dada2.qza
Saved FeatureData[Sequence] to: rep-seqs-dada2.qza
Saved SampleData[DADA2Stats] to: stats-dada2.qza


In [11]:
!qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv

Saved Visualization to: stats-dada2.qzv


In [12]:
Visualization.load('stats-dada2.qzv')

<visualization: Visualization uuid: 0b30d2f3-17e9-4dc9-820c-f4f253e54109>

Rename the output file

In [13]:
!mv rep-seqs-dada2.qza rep-seqs.qza
!mv table-dada2.qza table.qza

# FeatureTable and FeatureData summaries

After the quality filtering step completes, you’ll want to explore the resulting data. You can do this using the following two commands, which will create visual summaries of the data. 

In [14]:
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

Saved Visualization to: table.qzv


In [15]:
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

Saved Visualization to: rep-seqs.qzv


In [16]:
Visualization.load('rep-seqs.qzv')

<visualization: Visualization uuid: 8ce6a912-9164-4691-90e1-fed82e51304f>

# Generate a tree for phylogenetic diversity analyses

QIIME supports several phylogenetic diversity metrics, including Faith’s Phylogenetic Diversity and weighted and unweighted UniFrac. In addition to counts of features per sample (i.e., the data in the FeatureTable[Frequency] QIIME 2 artifact), these metrics require a rooted phylogenetic tree relating the features to one another.

터미널에서 다음 컴맨드입력, 주피터노트상에서는 오류가 발생

```bash
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza
```

```
Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza
```

# Alpha and beta diversity analysis

QIIME 2’s diversity analyses are available through the q2-diversity plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations.

In [22]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 1103 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jacc

After computing diversity metrics, we can begin to explore the microbial composition of the samples in the context of the sample metadata. 

In [23]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv


In [24]:
Visualization.load('core-metrics-results/faith-pd-group-significance.qzv')

<visualization: Visualization uuid: 56c69e52-98a1-44cf-9f2b-cd1ce220f13b>

In this data set, no continuous sample metadata columns (e.g., days-since-experiment-start) are correlated with alpha diversity, so we won’t test for those associations here

Next we’ll analyze sample composition in the context of categorical metadata using PERMANOVA (first described in Anderson (2001)) using the beta-group-significance command. 

In [25]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column body-site \
  --o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-body-site-significance.qzv


In [26]:
Visualization.load('core-metrics-results/unweighted-unifrac-body-site-significance.qzv')

<visualization: Visualization uuid: 22d6fd58-b980-4215-92c9-32dfd11ab887>

Again, none of the continuous sample metadata that we have for this data set are correlated with sample composition, so we won’t test for those associations here.

Finally, ordination is a popular approach for exploring microbial community composition in the context of sample metadata.



In [27]:
!qiime emperor plot \
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv

Saved Visualization to: core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv


In [28]:
Visualization.load('core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv')

<visualization: Visualization uuid: a9023d6a-eca6-4b33-ac4d-2e96eb481b52>

We will generate Emperor plots for unweighted UniFrac so that the resulting plot will contain axes for principal coordinate 1, principal coordinate 2, and days since the experiment start. We will use that last axis to explore how these samples changed over time.

# Alpha rarefaction plotting

In this section we’ll explore alpha diversity as a function of sampling depth using the qiime diversity alpha-rarefaction visualizer. 

In [29]:
!qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 4000 \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization alpha-rarefaction.qzv

Saved Visualization to: alpha-rarefaction.qzv


In [30]:
Visualization.load('alpha-rarefaction.qzv')

<visualization: Visualization uuid: d761115c-27f0-4ed8-b132-22575671aa4d>

The visualization will have two plots. 

- The top plot is an alpha rarefaction plot, and is primarily used to determine if the richness of the samples has been fully observed or sequenced. 
- The bottom plot in this visualization is important when grouping samples by metadata. It illustrates the number of samples that remain in each group when the feature table is rarefied to each sampling depth.

# Taxonomic analysis

In the next sections we’ll begin to explore the taxonomic composition of the samples, and again relate that to sample metadata.

In [31]:
!wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza"

--2019-09-05 10:28:00--  https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza [following]
--2019-09-05 10:28:01--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.234.104
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.234.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28373760 (27M) [application/x-www-form-urlencoded]
Saving to: `gg-13-8-99-515-806-nb-classifier.qza'

100%[======================================>] 28,373,760  3.04M/s   in 8.8s    

2019-09-05 10:28:11 (3.07 MB/s) - `gg-13-

In [32]:
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

Saved FeatureData[Taxonomy] to: taxonomy.qza


In [33]:
!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved Visualization to: taxonomy.qzv


Next, we can view the taxonomic composition of our samples with interactive bar plots. Generate those plots with the following command and then open the visualization.

In [34]:
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

Saved Visualization to: taxa-bar-plots.qzv


In [35]:
Visualization.load('taxa-bar-plots.qzv')

<visualization: Visualization uuid: 0ca17d3f-4089-470a-9b99-58a328de3dea>

# Differential abundance testing with ANCOM

ANCOM can be applied to identify features that are differentially abundant (i.e. present in different abundances) across sample groups.

We’ll start by creating a feature table that contains only the gut samples. 

In [36]:
!qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "[body-site]='gut'" \
  --o-filtered-table gut-table.qza

Saved FeatureTable[Frequency] to: gut-table.qza


ANCOM operates on a FeatureTable[Composition] QIIME 2 artifact, which is based on frequencies of features on a per-sample basis, but cannot tolerate frequencies of zero.

In [37]:
!qiime composition add-pseudocount \
  --i-table gut-table.qza \
  --o-composition-table comp-gut-table.qza

Saved FeatureTable[Composition] to: comp-gut-table.qza


We can then run ANCOM on the subject column to determine what features differ in abundance across the gut samples of the two subjects.

In [38]:
!qiime composition ancom \
  --i-table comp-gut-table.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization ancom-subject.qzv

Saved Visualization to: ancom-subject.qzv


In [39]:
Visualization.load('ancom-subject.qzv')

<visualization: Visualization uuid: c69706ea-a118-4f8b-b4e7-cbdfcebc0abf>

We’re also often interested in performing a differential abundance test at a specific taxonomic level.

In [40]:
!qiime taxa collapse \
  --i-table gut-table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table gut-table-l6.qza

Saved FeatureTable[Frequency] to: gut-table-l6.qza


In [41]:
!qiime composition add-pseudocount \
  --i-table gut-table-l6.qza \
  --o-composition-table comp-gut-table-l6.qza

Saved FeatureTable[Composition] to: comp-gut-table-l6.qza


In [42]:
!qiime composition ancom \
  --i-table comp-gut-table-l6.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization l6-ancom-subject.qzv

Saved Visualization to: l6-ancom-subject.qzv


In [43]:
Visualization.load('l6-ancom-subject.qzv')

<visualization: Visualization uuid: b2f1b137-80e5-42e8-8979-dfee845b8987>

## Question

Which genera differ in abundance across subject? In which subject is each genus more abundant?